In [1]:
import lancedb
import pyarrow as pa
import pandas as pd
from datasets import load_dataset
from transformers import GPT2Tokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Para este ejercicio descargamos una base de datos de mensajes de texto en inglés.

In [3]:
dataset = load_dataset("chirunder/text_messages")

Generating train split: 100%|██████████| 11615290/11615290 [00:21<00:00, 549933.03 examples/s] 


In [5]:
df = pd.DataFrame(dataset['train'])
df.rename(columns={'text': 'texto'}, inplace=True)
df.head()

,texto
0,Top right I gained a little speed with the add...
1,They are heavier wheels though as are all the ...
2,Federally registering a trademark is more than...
3,I'll have to jog my memory from rooting a few ...
4,Unless you can afford to buy all new larger cl...


Para los siguientes ejercicios, voy a crear una variable del numero de palabras en cada mensaje de texto.

In [9]:
df['n'] = df['texto'].apply(lambda x: len(str(x).split()))
df = df[['n', 'texto']]
df.head()

,n,texto
0,13,Top right I gained a little speed with the add...
1,14,They are heavier wheels though as are all the ...
2,9,Federally registering a trademark is more than...
3,21,I'll have to jog my memory from rooting a few ...
4,10,Unless you can afford to buy all new larger cl...


## Task 1:
A partir del dataframe df, crea df_tokenized (usando el Tokenizer de GPT2) con dos columnas pero con el texto tokenizado. Asegurate de que todos los embeddings tengan la misma longitud y los tokens sean enteros (todos enteros o todos doubles). 

In [ ]:
tokenizer = GPT2Tokenizer.from_pretained('gpt2')

tokenizer.pad_token = tokenier.eos_token

def tokenize(text):
    res = tokenizer(text)["input_ids"]
    if len(res) < 300:
        res = res + [tokenizer.pad_token_id] * (300 - len(res))
    else:
        res = res[:300]
    return res

df['texto_tokenizado'] = df['texto'].apply(tokenize)

df_fin = pd.DataFrame({'vector': df['texto_tokenizado'], 'texto': df['texto']})

df_fin.head()

## Task 2:
Mete el dataframe a una tabla en una base de datos de LanceDB.

In [ ]:
# Conectarse a la base de datos local
db = lancedb.connect("./.lancedb")

# Convertir el DataFrame de pandas a un objeto Table de LanceDB
table = lancedb.from_pandas(df_tokenized)

# Crear una tabla en la base de datos
db.create_table("tabla_tarea", schema=table.schema, primary_key='id')
db.insert("tabla_tarea", table)

## Task 3:
Haz una query estilo SQL a la tabla de la base de datos. Quiero que escribas la query equivalente y pongas la explicación de lo que está haciendo la consulta. Hint: usa la columna "n". 

In [ ]:
result = (db["tabla_tarea"].search()
    .where("texto LIKE '%love you%'")  # Usamos LIKE y los comodines '%' para buscar 'love you' en cualquier parte del texto
    .select(["texto", "n"])  # Seleccionamos solo las columnas 'texto' y 'n'
    .limit(11)  # Limitamos los resultados a 11 registros
    .to_pandas())  # Convertimos el resultado a un DataFrame de pandas

print(result)

- Query en SQL equivalente:

SELECT texto, n 

FROM tabla_tarea 

WHERE texto LIKE '%love you%' 

LIMIT 11;

- Explicacion: La consulta SQL selecciona columnas "texto" y "n" de "tabla_tarea" donde "texto" contiene "te amo", limitando los resultados a 11 registros para optimizar el manejo de datos.

## Task 4:
Inventa un mensaje de texto que tu podrías escribirle a un amigo. Tokenizalo y ponlo en el formato adecuado para hacer un vector query. Quiero que me regreses el mensaje más parecido al mensaje que inventaste (OJO: quiero el texto, no el embedding). HINT: Hay que decodear el resultado del query.

In [10]:
mensaje = 'Hi man. How you do doing? Do you know if we have to send the lanceDB homework today? Didn´t pay attention in class :('
mensaje_tokenizado = tokenize(mensaje)

In [ ]:
parecido_df = (db["tabla_tarea"].search(mensaje_tokenizado)
    .metric("cosine") # Puede ser L2 o cosine
    .select("texto")
    .limit(3)
    .to_pandas())
mensaje = parecido_df['texto'].values[0]
print(mensaje)

In [9]:
data = {'texto': ['Mi nombre es ...', 'Adios'], 'vector': [121323,123123]}
df = pd.DataFrame(data)
df

,texto,vector
0,Mi nombre es ...,121323
1,Adios,123123


In [10]:
mensa = data['texto'].values[0]
print(mensa)

AttributeError: 'list' object has no attribute 'values'